In [ ]:
# Try to build you code based on this one, so it will be easier for us to intergate all the codes into one file
# Create a function for each plot that you want to call
# Replace the screenshot in final report to the function you want to call 

In [ ]:
import pandas as pd
import numpy as np
import math

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Import data
# This cell should applies to the code of all of us
data_path = "./data/"
subscriptions = pd.read_csv(data_path + 'subscriptionsdataFINAL.csv') 
visitation_part1 = pd.read_csv(data_path + 'visitationdataPART1.csv') 
visitation_part2 = pd.read_csv(data_path + 'visitationdataPART2.csv') #convert the join and end date to_date
subscriptions['Join Date'] =  pd.to_datetime(subscriptions['Join Date'].str.upper(), format='%b-%y', yearfirst=False)
subscriptions['Join_year'] = pd.DatetimeIndex(subscriptions['Join Date']).year
subscriptions['End Date'] =  pd.to_datetime(subscriptions['End Date'].str.upper(), format='%b-%y', yearfirst=False)
subscriptions.dropna()

In [ ]:
# Create the seperate dataset to avoid any variable name conflict 
subscriptions_Qian =  subscriptions.copy()
visitation_part1_Qian = visitation_part1.copy() 
visitation_part2_Qian = visitation_part2.copy()  

In [ ]:
# Data Processing - Qian
df = subscriptions_Qian.copy()
numbers_club = pd.DataFrame(df.groupby('Join_year')['Club'].nunique())
numbers_club['Growth']=numbers_club['Club'].pct_change()
member_joined_data = subscriptions.groupby(['Join Date','Subscription Type']).size().reset_index(name='Number_of_member_joined')
member_joined_data = member_joined_data.rename(columns={'Join Date': "Date"})
member_left_data = subscriptions.groupby(['End Date','Subscription Type']).size().reset_index(name='Number_of_member_left')
member_left_data=member_left_data.rename(columns={'End Date': "Date"})
member_left_data['Date']=member_left_data['Date'] + pd.DateOffset(months=1)
member_data = pd.merge(member_joined_data,member_left_data,on = ['Subscription Type','Date'],how='outer').sort_index().replace({np.nan:0})
member_data['net_member'] = member_data['Number_of_member_joined'] - member_data['Number_of_member_left'] 
member_data = member_data.drop(columns = ['Number_of_member_joined', 'Number_of_member_left'])
Cumulative_Members = pd.DataFrame(member_data.groupby(['Date']).sum('net_member').unstack(fill_value=0))
Cumulative_Members = Cumulative_Members.rename(columns={0: "Cumulative_members"})
Cumulative_Members = Cumulative_Members.cumsum().reset_index()
Cumulative_Members_yearly = Cumulative_Members.copy()
Cumulative_Members_yearly['year'] = pd.DatetimeIndex(Cumulative_Members_yearly['Date']).year
Cumulative_Members_yearly=Cumulative_Members_yearly.groupby(['year']).sum()
Cumulative_Members_yearly=Cumulative_Members_yearly.drop([2019])
Cumulative_Members_yearly['Cumulative_members_Grwoth']=Cumulative_Members_yearly['Cumulative_members'].pct_change()
Cumulative_Members_seperating = pd.DataFrame(member_data.groupby(['Date','Subscription Type']).sum('net_member').unstack(fill_value=0))
Cumulative_Members_seperating = Cumulative_Members_seperating.rename(columns={0: "Cumulative_members"})
Cumulative_Members_seperating = Cumulative_Members_seperating.cumsum().reset_index()
x = Cumulative_Members_seperating['net_member']
y = pd.DataFrame({'Date':Cumulative_Members_seperating['Date']})
y['Flexible']=x['Flexible']
y['Standard']=x['Standard']
Cumulative_Members_seperating=y
def calculate_revenue_flexible(Date,net_member):
    date_while_price_changed = pd.Timestamp('2016-01-01') 
    if Date < date_while_price_changed:
        return net_member * 25.99
    elif Date >= date_while_price_changed:
        return net_member * 30.99
def calculate_revenue_standard(Date,net_member):
    date_while_price_changed = pd.Timestamp('2016-01-01') 
    if Date < date_while_price_changed:
        return net_member * 20.99
    elif Date >= date_while_price_changed:
        return net_member * 26.99 
Cumulative_Members_seperating_revenue = Cumulative_Members_seperating.copy()
Cumulative_Members_seperating_revenue['Flexible_revenue']=Cumulative_Members_seperating_revenue.apply(lambda row: calculate_revenue_flexible(row['Date'], row['Flexible']),axis=1)
Cumulative_Members_seperating_revenue['Standard_revenue']=Cumulative_Members_seperating_revenue.apply(lambda row: calculate_revenue_standard(row['Date'], row['Standard']),axis=1)
Cumulative_Members_seperating_revenue['Total_revenue']=Cumulative_Members_seperating_revenue['Flexible_revenue'] + Cumulative_Members_seperating_revenue['Standard_revenue']
Cumulative_Members_seperating_revenue_yearly = Cumulative_Members_seperating_revenue.copy()
Cumulative_Members_seperating_revenue_yearly['year'] = pd.DatetimeIndex(Cumulative_Members_seperating_revenue_yearly['Date']).year
Cumulative_Members_seperating_revenue_yearly=Cumulative_Members_seperating_revenue_yearly.groupby(['year']).sum()
Cumulative_Members_seperating_revenue_yearly=Cumulative_Members_seperating_revenue_yearly.drop([2019])
Cumulative_Members_seperating_revenue_yearly['Revenue_grwoth']=Cumulative_Members_seperating_revenue_yearly['Total_revenue'].pct_change()

In [ ]:
# Function to plot graph - Qian
def plot_growth_of_new_members():
    Year = Cumulative_Members_yearly.index.values
    Cumulative_members_Grwoth = Cumulative_Members_yearly['Cumulative_members_Grwoth']
    plt.plot(Year,Cumulative_members_Grwoth)
    plt.ylabel('%')
    plt.title('Growth of New Members')
def plot_cumulative_active_members():
    Date = Cumulative_Members.Date
    Cumulative_members = Cumulative_Members.Cumulative_members
    plt.plot(Date[:-1],Cumulative_members[:-1])
    plt.title('Cumulative Active Members')
    plt.axvline(x='2016-01-01')
def plot_monthly_revenue():
    Date = Cumulative_Members_seperating_revenue['Date']
    Total = Cumulative_Members_seperating_revenue['Total_revenue']
    plt.plot(Date[:-1],Total[:-1],label = 'Total')
    plt.axvline(x='2016-01-01')
    plt.ylabel('Pound')
    plt.title('Monthly Revenue')
def plot_growth_yearly_revenue():
    Date = Cumulative_Members_seperating_revenue_yearly.index.values
    Revenue_grwoth = Cumulative_Members_seperating_revenue_yearly['Revenue_grwoth']
    plt.plot(Date,Revenue_grwoth)
    plt.ylabel('%')
    plt.title('Yearly Revenue Growth')
def plot_bar_yearly_revenue():
    r =  np.arange(2009,2019)
    Total_revenue = Cumulative_Members_seperating_revenue_yearly['Total_revenue']
    plt.ylabel('Pound')
    plt.bar(r,Total_revenue)
    plt.title('Yearly Revenue')